This notebook is purely for data cleaning.  I have placed my raw data into a PostgreSQL database consisting of 5 tables:

1. Flights - detailed info about each flight in the United States in 2015
2. Airports - just a simple table linking the airport code to the full name
3. Airlines - just a simple table linking the airline code to the full name
4. Aircraft - information about each plane scraped from airfleets.net
5. Weather - daily weather summaries from the NOAA

I connect to the database using psycopg2 and then use SQL and pandas to get clean the data and engineer features.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import psycopg2 as pg2 
%matplotlib inline
sns.set()

password = 'hibbard'
host = 'localhost'
port = 5433

In [2]:
conn = pg2.connect(database='flights',user='postgres',password=password,host=host,port=port)

In [3]:
query = '''
SELECT * FROM aircraft LIMIT 10;
'''

pd.read_sql(query,conn)

,tail_num,manufacturer,model,engine,first_flight
0,N101DU,Airbus,A220,2 x PW PW1524G,06/10/2018
1,N102DU,Airbus,A220,2 x PW PW1524G,07/12/2018
2,N103DU,Airbus,A220,2 x PW PW1524G,22/11/2018
3,N104DU,Airbus,A220,2 x PW PW1524G,20/12/2018
4,N105DU,Airbus,A220,2 x PW PW1524G,03/02/2019
5,N106DU,Airbus,A220,2 x PW PW1524G,09/02/2019
6,N107DU,Airbus,A220,2 x PW PW1524G,12/02/2019
7,N108DQ,Airbus,A220,2 x PW PW1524G,12/04/2019
8,N109DU,Airbus,A220,2 x PW PW1524G,23/02/2019
9,N110DU,Airbus,A220,2 x PW PW1524G,16/03/2019


In [4]:
query = '''
SELECT * FROM airlines LIMIT 10;
'''

pd.read_sql(query,conn)

,iata_code,airline
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [5]:
query = '''
SELECT * FROM airports;
'''

ports = pd.read_sql(query,conn)

In [6]:
query = '''
SELECT * FROM flights LIMIT 10;
'''

pd.read_sql(query,conn)

,year,month,day,day_of_week,airline,flight_num,tail_num,orig_airport,dest_airport,sched_depart,...,arr_time,arr_delay,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,2015,1,4,7,US,2075,N713UW,CLT,BNA,2225,...,2247.0,2.0,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2015,1,4,7,US,2086,N712US,CLT,RIC,2225,...,2325.0,-2.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,2015,1,4,7,US,864,N806AW,CLT,ORF,2225,...,2329.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,2015,1,4,7,US,2035,N737US,CLT,JAX,2225,...,2332.0,-8.0,0,0,None,NaN,NaN,NaN,NaN,NaN
4,2015,1,4,7,AA,2497,N3ABAA,DFW,LAX,2225,...,21.0,31.0,0,0,None,14.0,0.0,16.0,1.0,0.0
5,2015,1,4,7,AS,473,N794AS,LAX,SEA,2225,...,102.0,-4.0,0,0,None,NaN,NaN,NaN,NaN,NaN
6,2015,1,4,7,B6,1602,N651JB,FLL,JFK,2225,...,147.0,45.0,0,0,None,0.0,0.0,9.0,36.0,0.0
7,2015,1,4,7,F9,1352,N216FR,IAD,STL,2225,...,5.0,20.0,0,0,None,6.0,0.0,14.0,0.0,0.0
8,2015,1,4,7,F9,1102,N912FR,ATL,CLE,2225,...,27.0,12.0,0,0,None,NaN,NaN,NaN,NaN,NaN
9,2015,1,4,7,NK,709,N502NK,LAX,DTW,2225,...,709.0,93.0,0,0,None,93.0,0.0,0.0,0.0,0.0


In [7]:
query = '''
SELECT * FROM weather;
'''

weather = pd.read_sql(query,conn)

In [8]:
query = '''
SELECT * FROM aircraft  
INNER JOIN flights 
ON aircraft.tail_num = flights.tail_num
INNER JOIN airlines
ON airlines.iata_code = flights.airline ;
'''

air = pd.read_sql(query,conn)
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005837 entries, 0 to 3005836
Data columns (total 38 columns):
 #   Column               Dtype  
---  ------               -----  
 0   tail_num             object 
 1   manufacturer         object 
 2   model                object 
 3   engine               object 
 4   first_flight         object 
 5   year                 int64  
 6   month                int64  
 7   day                  int64  
 8   day_of_week          int64  
 9   airline              object 
 10  flight_num           int64  
 11  tail_num             object 
 12  orig_airport         object 
 13  dest_airport         object 
 14  sched_depart         int64  
 15  depart_time          float64
 16  depart_delay         float64
 17  taxi_out             float64
 18  wheels_off           float64
 19  sched_time           float64
 20  elapsed_time         float64
 21  air_time             float64
 22  distance             int64  
 23  wheels_on            float64
 24

In [9]:
air.head()

,tail_num,manufacturer,model,engine,first_flight,year,month,day,day_of_week,airline,...,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,iata_code,airline
0,N713UW,Airbus,A319,2 x CFMI CFM56-5B6/P,17/06/1999,2015,1,4,7,US,...,0,0,None,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
1,N712US,Airbus,A319,2 x CFMI CFM56-5B6/P,03/06/1999,2015,1,4,7,US,...,0,0,None,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
2,N806AW,Airbus,A319,2 x IAE V2524-A5,05/07/1999,2015,1,4,7,US,...,0,0,None,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
3,N737US,Airbus,A319,2 x CFMI CFM56-5B6/P,20/05/2000,2015,1,4,7,US,...,0,0,None,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
4,N651JB,Airbus,A320,2 x IAE V2527-A5,07/12/2006,2015,1,4,7,B6,...,0,0,None,0.0,0.0,9.0,36.0,0.0,B6,JetBlue Airways


In [10]:
# raw data provides a year, month and day of flight - convert that to datetime
air['flight_date'] = pd.to_datetime(air[['year','month','day']])

In [11]:
air['flight_date'] = air.flight_date.dt.date

In [12]:
weather['date'] = pd.to_datetime(weather['date'])
weather['date'] = weather.date.dt.date

Merge everything into one dataframe where each row is a unique flight and contains all the necessary information to predict a flight delay.

In [13]:
df1 = pd.merge(air,weather,left_on=['orig_airport','flight_date'],
              right_on=['airport_code','date'])

In [14]:
df2 = pd.merge(df1,weather,left_on=['dest_airport','flight_date'],
              right_on=['airport_code','date'])

In [15]:
df3 = pd.merge(df2,ports,left_on='orig_airport',right_on='iata_code')

In [16]:
df_raw = pd.merge(df3,ports,left_on='dest_airport',right_on='iata_code')

In [17]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1580818 entries, 0 to 1580817
Data columns (total 71 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   tail_num             1580818 non-null  object 
 1   manufacturer         1580818 non-null  object 
 2   model                1580818 non-null  object 
 3   engine               1580818 non-null  object 
 4   first_flight         1580818 non-null  object 
 5   year                 1580818 non-null  int64  
 6   month                1580818 non-null  int64  
 7   day                  1580818 non-null  int64  
 8   day_of_week          1580818 non-null  int64  
 9   airline              1580818 non-null  object 
 10  flight_num           1580818 non-null  int64  
 11  tail_num             1580818 non-null  object 
 12  orig_airport         1580818 non-null  object 
 13  dest_airport         1580818 non-null  object 
 14  sched_depart         1580818 non-null  int64  
 15

In [18]:
df_raw.columns

Index(['tail_num', 'manufacturer', 'model', 'engine', 'first_flight', 'year',
       'month', 'day', 'day_of_week', 'airline', 'flight_num', 'tail_num',
       'orig_airport', 'dest_airport', 'sched_depart', 'depart_time',
       'depart_delay', 'taxi_out', 'wheels_off', 'sched_time', 'elapsed_time',
       'air_time', 'distance', 'wheels_on', 'taxi_in', 'sched_arr', 'arr_time',
       'arr_delay', 'diverted', 'cancelled', 'cancel_reason', 'air_sys_delay',
       'security_delay', 'airline_delay', 'late_aircraft_delay',
       'weather_delay', 'iata_code_x', 'airline', 'flight_date', 'station_x',
       'name_x', 'date_x', 'avg_wind_x', 'precip_x', 'snow_x', 'max_temp_x',
       'min_temp_x', 'airport_code_x', 'station_y', 'name_y', 'date_y',
       'avg_wind_y', 'precip_y', 'snow_y', 'max_temp_y', 'min_temp_y',
       'airport_code_y', 'iata_code_y', 'airport_x', 'city_x', 'state_x',
       'country_x', 'lat_x', 'long_x', 'iata_code', 'airport_y', 'city_y',
       'state_y', 'country_

In [19]:
# rename some of the columns so I know which ones are duplicates and can be dropped

df_raw.columns = ['tail_num', 'manufacturer', 'model', 'engine', 'first_flight', 'year',
       'month', 'day', 'day_of_week', 'airline_drop', 'flight_num', 'tail_num_drop',
       'orig_airport', 'dest_airport', 'sched_depart', 'depart_time',
       'depart_delay', 'taxi_out', 'wheels_off', 'sched_time', 'elapsed_time',
       'air_time', 'distance', 'wheels_on', 'taxi_in', 'sched_arr', 'arr_time',
       'arr_delay', 'diverted', 'cancelled', 'cancel_reason', 'air_sys_delay',
       'security_delay', 'airline_delay', 'late_aircraft_delay',
       'weather_delay', 'iata_code_x', 'airline', 'flight_date', 'station_orig',
       'name_orig', 'date_x', 'avg_wind_orig', 'precip_orig', 'snow_orig', 'max_temp_orig',
       'min_temp_orig', 'airport_code_x', 'station_dest', 'name_dest', 'date_y',
       'avg_wind_dest', 'precip_dest', 'snow_dest', 'max_temp_dest', 'min_temp_dest',
       'airport_code_dest', 'iata_code_orig', 'airport_name_orig', 'city_orig', 'state_orig',
       'country_orig', 'lat_orig', 'long_orig', 'iata_code_dest', 'airport_dest', 'city_dest',
       'state_dest', 'country_dest', 'lat_dest', 'long_dest']

In [20]:
pd.options.display.max_columns = None

In [21]:
df_raw.drop(['tail_num_drop','airline_drop','date_x','date_y'],axis=1,inplace=True)

In [22]:
df_raw.head()

,tail_num,manufacturer,model,engine,first_flight,year,month,day,day_of_week,flight_num,orig_airport,dest_airport,sched_depart,depart_time,depart_delay,taxi_out,wheels_off,sched_time,elapsed_time,air_time,distance,wheels_on,taxi_in,sched_arr,arr_time,arr_delay,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,iata_code_x,airline,flight_date,station_orig,name_orig,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,airport_code_x,station_dest,name_dest,avg_wind_dest,precip_dest,snow_dest,max_temp_dest,min_temp_dest,airport_code_dest,iata_code_orig,airport_name_orig,city_orig,state_orig,country_orig,lat_orig,long_orig,iata_code_dest,airport_dest,city_dest,state_dest,country_dest,lat_dest,long_dest
0,N502NK,Airbus,A319,2 x IAE V2524-A5,11/03/2005,2015,1,4,7,709,LAX,DTW,2225,2338.0,73.0,11.0,2349.0,251.0,271.0,247.0,1979,656.0,13.0,536.0,709.0,93.0,0,0,None,93.0,0.0,0.0,0.0,0.0,NK,Spirit Air Lines,2015-01-04,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",4.70,0.0,0.0,61.0,42.0,LAX,USW00094847,"DETROIT METRO AIRPORT, MI US",11.86,0.12,0.8,36.0,19.0,DTW,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,DTW,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
1,N372DA,Boeing,737,2 x CFMI CFM56-7B26,23/09/1998,2015,1,4,7,1806,LAX,DTW,928,944.0,16.0,15.0,959.0,267.0,262.0,238.0,1979,1657.0,9.0,1655.0,1706.0,11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",4.70,0.0,0.0,61.0,42.0,LAX,USW00094847,"DETROIT METRO AIRPORT, MI US",11.86,0.12,0.8,36.0,19.0,DTW,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,DTW,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
2,N550NW,Boeing,757,2 x PW PW2037,02/06/2001,2015,1,4,7,1706,LAX,DTW,1135,1159.0,24.0,21.0,1220.0,269.0,268.0,239.0,1979,1919.0,8.0,1904.0,1927.0,23.0,0,0,None,0.0,0.0,19.0,4.0,0.0,DL,Delta Air Lines Inc.,2015-01-04,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",4.70,0.0,0.0,61.0,42.0,LAX,USW00094847,"DETROIT METRO AIRPORT, MI US",11.86,0.12,0.8,36.0,19.0,DTW,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,DTW,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
3,N693DL,Boeing,757,2 x PW PW2037,27/09/1998,2015,1,4,7,870,LAX,DTW,1330,1330.0,0.0,21.0,1351.0,261.0,262.0,232.0,1979,2043.0,9.0,2051.0,2052.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",4.70,0.0,0.0,61.0,42.0,LAX,USW00094847,"DETROIT METRO AIRPORT, MI US",11.86,0.12,0.8,36.0,19.0,DTW,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,DTW,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
4,N6706Q,Boeing,757,2 x PW PW2037,21/03/2000,2015,1,5,1,1876,LAX,DTW,1330,1328.0,-2.0,17.0,1345.0,267.0,268.0,237.0,1979,2042.0,14.0,2057.0,2056.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-05,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",5.59,0.0,0.0,79.0,50.0,LAX,USW00094847,"DETROIT METRO AIRPORT, MI US",13.65,0.00,0.2,19.0,8.0,DTW,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,DTW,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884


In [23]:
df_raw.drop(['airport_code_x','airport_code_dest','iata_code_orig','iata_code_dest',
            'station_orig','station_dest','name_orig','name_dest'],axis=1,inplace=True)

In [24]:
df_raw.head()

,tail_num,manufacturer,model,engine,first_flight,year,month,day,day_of_week,flight_num,orig_airport,dest_airport,sched_depart,depart_time,depart_delay,taxi_out,wheels_off,sched_time,elapsed_time,air_time,distance,wheels_on,taxi_in,sched_arr,arr_time,arr_delay,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,iata_code_x,airline,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,avg_wind_dest,precip_dest,snow_dest,max_temp_dest,min_temp_dest,airport_name_orig,city_orig,state_orig,country_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,country_dest,lat_dest,long_dest
0,N502NK,Airbus,A319,2 x IAE V2524-A5,11/03/2005,2015,1,4,7,709,LAX,DTW,2225,2338.0,73.0,11.0,2349.0,251.0,271.0,247.0,1979,656.0,13.0,536.0,709.0,93.0,0,0,None,93.0,0.0,0.0,0.0,0.0,NK,Spirit Air Lines,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
1,N372DA,Boeing,737,2 x CFMI CFM56-7B26,23/09/1998,2015,1,4,7,1806,LAX,DTW,928,944.0,16.0,15.0,959.0,267.0,262.0,238.0,1979,1657.0,9.0,1655.0,1706.0,11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
2,N550NW,Boeing,757,2 x PW PW2037,02/06/2001,2015,1,4,7,1706,LAX,DTW,1135,1159.0,24.0,21.0,1220.0,269.0,268.0,239.0,1979,1919.0,8.0,1904.0,1927.0,23.0,0,0,None,0.0,0.0,19.0,4.0,0.0,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
3,N693DL,Boeing,757,2 x PW PW2037,27/09/1998,2015,1,4,7,870,LAX,DTW,1330,1330.0,0.0,21.0,1351.0,261.0,262.0,232.0,1979,2043.0,9.0,2051.0,2052.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
4,N6706Q,Boeing,757,2 x PW PW2037,21/03/2000,2015,1,5,1,1876,LAX,DTW,1330,1328.0,-2.0,17.0,1345.0,267.0,268.0,237.0,1979,2042.0,14.0,2057.0,2056.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-05,5.59,0.0,0.0,79.0,50.0,13.65,0.00,0.2,19.0,8.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884


In [25]:
df_raw.drop('year',axis=1,inplace=True)

In [26]:
df_raw.columns = ['tail_num', 'manufacturer', 'model', 'engine', 'first_flight', 'month',
       'day', 'day_of_week', 'flight_num', 'orig_airport', 'dest_airport',
       'sched_depart', 'depart_time', 'depart_delay', 'taxi_out', 'wheels_off',
       'sched_time', 'elapsed_time', 'air_time', 'distance', 'wheels_on',
       'taxi_in', 'sched_arr', 'arr_time', 'arr_delay', 'diverted',
       'cancelled', 'cancel_reason', 'air_sys_delay', 'security_delay',
       'airline_delay', 'late_aircraft_delay', 'weather_delay', 'airline_code',
       'airline_name', 'flight_date', 'avg_wind_orig', 'precip_orig', 'snow_orig',
       'max_temp_orig', 'min_temp_orig', 'avg_wind_dest', 'precip_dest',
       'snow_dest', 'max_temp_dest', 'min_temp_dest', 'airport_name_orig',
       'city_orig', 'state_orig', 'country_orig', 'lat_orig', 'long_orig',
       'airport_dest', 'city_dest', 'state_dest', 'country_dest', 'lat_dest',
       'long_dest']

Engine information is a string which typically indicates the number of engines and model in an odd format.  I will extract just the manufacturer abbreviation, keep that as a column, and add in a translation to the full name as well.

In [27]:
df_raw[['num','engine_a']] = df_raw.engine.str.split(' x ',expand=True)

In [28]:
df_raw.drop(['engine','num'],axis=1,inplace=True)

In [29]:
df_raw[['engine_manufacturer','a','b']] = df_raw.engine_a.str.split(' ',expand=True)

In [30]:
df_raw.drop(['a','b'],axis=1,inplace=True)

In [31]:
df_raw.engine_manufacturer.unique()

array(['IAE', 'CFMI', 'PW', 'GE', '', 'RR', 'BMW', 'Genx-1b74/75/P1',
       'GEnx-1B'], dtype=object)

In [32]:
df_raw.engine_manufacturer.replace('Genx-1b74/75/P1','GE',inplace=True)
df_raw.engine_manufacturer.replace('GEnx-1B','GE',inplace=True)

In [33]:
df_raw.engine_manufacturer.unique()

array(['IAE', 'CFMI', 'PW', 'GE', '', 'RR', 'BMW'], dtype=object)

In [34]:
df_raw.engine_manufacturer.value_counts()

CFMI    1033516
IAE      293573
PW       118932
BMW       81778
RR        29193
GE        16655
           7171
Name: engine_manufacturer, dtype: int64

In [35]:
df_raw = df_raw[df_raw.engine_manufacturer!='']

In [36]:
df_raw.drop('engine_a',axis=1,inplace=True)

The raw data from airfleets.net provided the date of each aircraft's first flight, which I translate into an age of the aircraft in years.

In [37]:
# last 4 digits of the string date is the year
df_raw['year_plane_made'] = df_raw.first_flight.str[-4:]

In [38]:
df_raw.year_plane_made.value_counts()

2001    141016
2000    135319
1999    101414
2006     96016
1998     87627
2004     75235
2002     75220
2007     73130
2005     69367
2008     66516
nown     66495
2012     58561
2003     54918
2014     48658
2011     41703
2009     38873
2013     35688
1997     30500
1991     29829
1992     29669
2010     28404
1993     28238
1994     28021
1995     24939
1996     24099
2015     23131
1990     15200
1988     14374
1987      9957
1986      7826
1985      7782
1989      3169
1984      2753
Name: year_plane_made, dtype: int64

In [39]:
df_raw = df_raw[df_raw.year_plane_made!='nown']

In [40]:
df_raw.head()

,tail_num,manufacturer,model,first_flight,month,day,day_of_week,flight_num,orig_airport,dest_airport,sched_depart,depart_time,depart_delay,taxi_out,wheels_off,sched_time,elapsed_time,air_time,distance,wheels_on,taxi_in,sched_arr,arr_time,arr_delay,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,airline_code,airline_name,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,avg_wind_dest,precip_dest,snow_dest,max_temp_dest,min_temp_dest,airport_name_orig,city_orig,state_orig,country_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,country_dest,lat_dest,long_dest,engine_manufacturer,year_plane_made
0,N502NK,Airbus,A319,11/03/2005,1,4,7,709,LAX,DTW,2225,2338.0,73.0,11.0,2349.0,251.0,271.0,247.0,1979,656.0,13.0,536.0,709.0,93.0,0,0,None,93.0,0.0,0.0,0.0,0.0,NK,Spirit Air Lines,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,IAE,2005
1,N372DA,Boeing,737,23/09/1998,1,4,7,1806,LAX,DTW,928,944.0,16.0,15.0,959.0,267.0,262.0,238.0,1979,1657.0,9.0,1655.0,1706.0,11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,CFMI,1998
2,N550NW,Boeing,757,02/06/2001,1,4,7,1706,LAX,DTW,1135,1159.0,24.0,21.0,1220.0,269.0,268.0,239.0,1979,1919.0,8.0,1904.0,1927.0,23.0,0,0,None,0.0,0.0,19.0,4.0,0.0,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,2001
3,N693DL,Boeing,757,27/09/1998,1,4,7,870,LAX,DTW,1330,1330.0,0.0,21.0,1351.0,261.0,262.0,232.0,1979,2043.0,9.0,2051.0,2052.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,1998
4,N6706Q,Boeing,757,21/03/2000,1,5,1,1876,LAX,DTW,1330,1328.0,-2.0,17.0,1345.0,267.0,268.0,237.0,1979,2042.0,14.0,2057.0,2056.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-05,5.59,0.0,0.0,79.0,50.0,13.65,0.00,0.2,19.0,8.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,2000


In [41]:
df_raw['sched_depart'] = df_raw.sched_depart.astype(str).str.zfill(4)

In [42]:
df_raw['hour'] = df_raw.sched_depart.str[:2]

In [43]:
df_raw.hour.unique()

array(['22', '09', '11', '13', '07', '23', '00', '21', '06', '14', '16',
       '15', '10', '08', '05', '12', '17', '19', '18', '20', '01', '02'],
      dtype=object)

In [44]:
df_raw.drop('sched_depart',axis=1,inplace=True)

In [45]:
df_raw.head()

,tail_num,manufacturer,model,first_flight,month,day,day_of_week,flight_num,orig_airport,dest_airport,depart_time,depart_delay,taxi_out,wheels_off,sched_time,elapsed_time,air_time,distance,wheels_on,taxi_in,sched_arr,arr_time,arr_delay,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,airline_code,airline_name,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,avg_wind_dest,precip_dest,snow_dest,max_temp_dest,min_temp_dest,airport_name_orig,city_orig,state_orig,country_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,country_dest,lat_dest,long_dest,engine_manufacturer,year_plane_made,hour
0,N502NK,Airbus,A319,11/03/2005,1,4,7,709,LAX,DTW,2338.0,73.0,11.0,2349.0,251.0,271.0,247.0,1979,656.0,13.0,536.0,709.0,93.0,0,0,None,93.0,0.0,0.0,0.0,0.0,NK,Spirit Air Lines,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,IAE,2005,22
1,N372DA,Boeing,737,23/09/1998,1,4,7,1806,LAX,DTW,944.0,16.0,15.0,959.0,267.0,262.0,238.0,1979,1657.0,9.0,1655.0,1706.0,11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,CFMI,1998,09
2,N550NW,Boeing,757,02/06/2001,1,4,7,1706,LAX,DTW,1159.0,24.0,21.0,1220.0,269.0,268.0,239.0,1979,1919.0,8.0,1904.0,1927.0,23.0,0,0,None,0.0,0.0,19.0,4.0,0.0,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,2001,11
3,N693DL,Boeing,757,27/09/1998,1,4,7,870,LAX,DTW,1330.0,0.0,21.0,1351.0,261.0,262.0,232.0,1979,2043.0,9.0,2051.0,2052.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,1998,13
4,N6706Q,Boeing,757,21/03/2000,1,5,1,1876,LAX,DTW,1328.0,-2.0,17.0,1345.0,267.0,268.0,237.0,1979,2042.0,14.0,2057.0,2056.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-05,5.59,0.0,0.0,79.0,50.0,13.65,0.00,0.2,19.0,8.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,2000,13


In [46]:
# data source provided numerous delay reasons, I want the aggregate total delay

df_raw['total_delay'] = df_raw.air_sys_delay + df_raw.security_delay + df_raw.airline_delay + df_raw.late_aircraft_delay + df_raw.weather_delay

In [47]:
# turn the continuous variable 'total delay' into a binary delayed/not delayed

df_raw['delayed'] = df_raw.total_delay>0

In [48]:
df_raw.delayed.value_counts()

False    1212612
True      294540
Name: delayed, dtype: int64

In [49]:
df_raw.head()

,tail_num,manufacturer,model,first_flight,month,day,day_of_week,flight_num,orig_airport,dest_airport,depart_time,depart_delay,taxi_out,wheels_off,sched_time,elapsed_time,air_time,distance,wheels_on,taxi_in,sched_arr,arr_time,arr_delay,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,airline_code,airline_name,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,avg_wind_dest,precip_dest,snow_dest,max_temp_dest,min_temp_dest,airport_name_orig,city_orig,state_orig,country_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,country_dest,lat_dest,long_dest,engine_manufacturer,year_plane_made,hour,total_delay,delayed
0,N502NK,Airbus,A319,11/03/2005,1,4,7,709,LAX,DTW,2338.0,73.0,11.0,2349.0,251.0,271.0,247.0,1979,656.0,13.0,536.0,709.0,93.0,0,0,None,93.0,0.0,0.0,0.0,0.0,NK,Spirit Air Lines,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,IAE,2005,22,93.0,True
1,N372DA,Boeing,737,23/09/1998,1,4,7,1806,LAX,DTW,944.0,16.0,15.0,959.0,267.0,262.0,238.0,1979,1657.0,9.0,1655.0,1706.0,11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,CFMI,1998,09,NaN,False
2,N550NW,Boeing,757,02/06/2001,1,4,7,1706,LAX,DTW,1159.0,24.0,21.0,1220.0,269.0,268.0,239.0,1979,1919.0,8.0,1904.0,1927.0,23.0,0,0,None,0.0,0.0,19.0,4.0,0.0,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,2001,11,23.0,True
3,N693DL,Boeing,757,27/09/1998,1,4,7,870,LAX,DTW,1330.0,0.0,21.0,1351.0,261.0,262.0,232.0,1979,2043.0,9.0,2051.0,2052.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,1998,13,NaN,False
4,N6706Q,Boeing,757,21/03/2000,1,5,1,1876,LAX,DTW,1328.0,-2.0,17.0,1345.0,267.0,268.0,237.0,1979,2042.0,14.0,2057.0,2056.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-05,5.59,0.0,0.0,79.0,50.0,13.65,0.00,0.2,19.0,8.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,2000,13,NaN,False


In [50]:
df_raw['aircraft_age'] = 2015 - df_raw.year_plane_made.astype(int)

In [51]:
df_raw.drop(['year_plane_made','first_flight'],axis=1,inplace=True)

In [52]:
df_raw.aircraft_age.value_counts()

14    141016
15    135319
16    101414
9      96016
17     87627
11     75235
13     75220
8      73130
10     69367
7      66516
3      58561
12     54918
1      48658
4      41703
6      38873
2      35688
18     30500
24     29829
23     29669
5      28404
22     28238
21     28021
20     24939
19     24099
0      23131
25     15200
27     14374
28      9957
29      7826
30      7782
26      3169
31      2753
Name: aircraft_age, dtype: int64

In [53]:
df_raw.head()

,tail_num,manufacturer,model,month,day,day_of_week,flight_num,orig_airport,dest_airport,depart_time,depart_delay,taxi_out,wheels_off,sched_time,elapsed_time,air_time,distance,wheels_on,taxi_in,sched_arr,arr_time,arr_delay,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,airline_code,airline_name,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,avg_wind_dest,precip_dest,snow_dest,max_temp_dest,min_temp_dest,airport_name_orig,city_orig,state_orig,country_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,country_dest,lat_dest,long_dest,engine_manufacturer,hour,total_delay,delayed,aircraft_age
0,N502NK,Airbus,A319,1,4,7,709,LAX,DTW,2338.0,73.0,11.0,2349.0,251.0,271.0,247.0,1979,656.0,13.0,536.0,709.0,93.0,0,0,None,93.0,0.0,0.0,0.0,0.0,NK,Spirit Air Lines,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,IAE,22,93.0,True,10
1,N372DA,Boeing,737,1,4,7,1806,LAX,DTW,944.0,16.0,15.0,959.0,267.0,262.0,238.0,1979,1657.0,9.0,1655.0,1706.0,11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,CFMI,09,NaN,False,17
2,N550NW,Boeing,757,1,4,7,1706,LAX,DTW,1159.0,24.0,21.0,1220.0,269.0,268.0,239.0,1979,1919.0,8.0,1904.0,1927.0,23.0,0,0,None,0.0,0.0,19.0,4.0,0.0,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,11,23.0,True,14
3,N693DL,Boeing,757,1,4,7,870,LAX,DTW,1330.0,0.0,21.0,1351.0,261.0,262.0,232.0,1979,2043.0,9.0,2051.0,2052.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,13,NaN,False,17
4,N6706Q,Boeing,757,1,5,1,1876,LAX,DTW,1328.0,-2.0,17.0,1345.0,267.0,268.0,237.0,1979,2042.0,14.0,2057.0,2056.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-05,5.59,0.0,0.0,79.0,50.0,13.65,0.00,0.2,19.0,8.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,13,NaN,False,15


In [54]:
# delete these because a flight would already be delayed by the time these are known
df_raw.drop(['depart_time','depart_delay','taxi_out','wheels_off','sched_time','elapsed_time',
            'air_time','wheels_on','taxi_in','sched_arr','arr_time','arr_delay'],axis=1,inplace=True)

In [55]:
df_raw.head()

,tail_num,manufacturer,model,month,day,day_of_week,flight_num,orig_airport,dest_airport,distance,diverted,cancelled,cancel_reason,air_sys_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,airline_code,airline_name,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,avg_wind_dest,precip_dest,snow_dest,max_temp_dest,min_temp_dest,airport_name_orig,city_orig,state_orig,country_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,country_dest,lat_dest,long_dest,engine_manufacturer,hour,total_delay,delayed,aircraft_age
0,N502NK,Airbus,A319,1,4,7,709,LAX,DTW,1979,0,0,None,93.0,0.0,0.0,0.0,0.0,NK,Spirit Air Lines,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,IAE,22,93.0,True,10
1,N372DA,Boeing,737,1,4,7,1806,LAX,DTW,1979,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,CFMI,09,NaN,False,17
2,N550NW,Boeing,757,1,4,7,1706,LAX,DTW,1979,0,0,None,0.0,0.0,19.0,4.0,0.0,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,11,23.0,True,14
3,N693DL,Boeing,757,1,4,7,870,LAX,DTW,1979,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-04,4.70,0.0,0.0,61.0,42.0,11.86,0.12,0.8,36.0,19.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,13,NaN,False,17
4,N6706Q,Boeing,757,1,5,1,1876,LAX,DTW,1979,0,0,None,NaN,NaN,NaN,NaN,NaN,DL,Delta Air Lines Inc.,2015-01-05,5.59,0.0,0.0,79.0,50.0,13.65,0.00,0.2,19.0,8.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,PW,13,NaN,False,15


In [56]:
df_raw.airline_name.value_counts()

Southwest Airlines Co.    586792
Delta Air Lines Inc.      289334
United Air Lines Inc.     247457
US Airways Inc.            67582
JetBlue Airways            64082
American Airlines Inc.     63753
Spirit Air Lines           62245
Alaska Airlines Inc.       52156
Virgin America             38345
Frontier Airlines Inc.     25073
Hawaiian Airlines Inc.     10333
Name: airline_name, dtype: int64

In [57]:
# drop US Airways and Virgin America because they no longer exist

df_raw = df_raw[df_raw.airline_name!='US Airways Inc.']
df_raw = df_raw[df_raw.airline_name!='Virgin America']

In [58]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1401225 entries, 0 to 1580817
Data columns (total 48 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   tail_num             1401225 non-null  object 
 1   manufacturer         1401225 non-null  object 
 2   model                1401225 non-null  object 
 3   month                1401225 non-null  int64  
 4   day                  1401225 non-null  int64  
 5   day_of_week          1401225 non-null  int64  
 6   flight_num           1401225 non-null  int64  
 7   orig_airport         1401225 non-null  object 
 8   dest_airport         1401225 non-null  object 
 9   distance             1401225 non-null  int64  
 10  diverted             1401225 non-null  object 
 11  cancelled            1401225 non-null  object 
 12  cancel_reason        11717 non-null    object 
 13  air_sys_delay        274584 non-null   float64
 14  security_delay       274584 non-null   float64
 15

In [59]:
# drop destination weather, diversions and cancellations (we're only predicting delays)
# drop sub-reasons for delay (we only need total delay)
# drop country because it's all USA

df_raw.drop(['avg_wind_dest','precip_dest','snow_dest','max_temp_dest','min_temp_dest',
            'diverted','cancelled','cancel_reason','air_sys_delay','security_delay',
            'airline_delay','late_aircraft_delay','weather_delay','country_orig',
            'country_dest'],axis=1,inplace=True)

Here I find, for each flight, was the exact aircraft to be used for that flight delayed on its previous flight, and was that flight on the same day? With the thinking that if the plane to be used for your flight was delayed earlier in the same day, the chances of your flight being delayed would be higher.  This turned out to be very true.

In [60]:
df_raw['flight_date'] = pd.to_datetime(df_raw.flight_date)

In [61]:
# sort by aircraft unique identifier and date for this to work
df_raw.sort_values(by=['tail_num','flight_date'],inplace=True)

In [62]:
df_raw.head(25)

,tail_num,manufacturer,model,month,day,day_of_week,flight_num,orig_airport,dest_airport,distance,airline_code,airline_name,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,airport_name_orig,city_orig,state_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,lat_dest,long_dest,engine_manufacturer,hour,total_delay,delayed,aircraft_age
769672,N102UW,Airbus,A320,7,21,2,1829,TPA,PHX,1788,AA,American Airlines Inc.,2015-07-21,8.50,0.01,0.0,89.0,81.0,Tampa International Airport,Tampa,FL,27.97547,-82.53325,Phoenix Sky Harbor International Airport,Phoenix,AZ,33.43417,-112.00806,CFMI,20,NaN,False,17
1228456,N102UW,Airbus,A320,7,21,2,575,PHX,TPA,1788,AA,American Airlines Inc.,2015-07-21,6.71,0.00,0.0,106.0,85.0,Phoenix Sky Harbor International Airport,Phoenix,AZ,33.43417,-112.00806,Tampa International Airport,Tampa,FL,27.97547,-82.53325,CFMI,23,NaN,False,17
708469,N102UW,Airbus,A320,7,22,3,1789,TPA,PHL,920,AA,American Airlines Inc.,2015-07-22,7.16,0.04,0.0,91.0,80.0,Tampa International Airport,Tampa,FL,27.97547,-82.53325,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,07,NaN,False,17
700195,N102UW,Airbus,A320,8,9,7,1850,MCO,PHL,861,AA,American Airlines Inc.,2015-08-09,6.26,0.00,0.0,94.0,74.0,Orlando International Airport,Orlando,FL,28.42889,-81.31603,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,10,NaN,False,17
1069062,N102UW,Airbus,A320,8,9,7,1850,PHL,MCO,861,AA,American Airlines Inc.,2015-08-09,4.70,0.00,0.0,86.0,71.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Orlando International Airport,Orlando,FL,28.42889,-81.31603,CFMI,07,NaN,False,17
1230079,N102UW,Airbus,A320,8,10,1,1883,PHL,TPA,920,AA,American Airlines Inc.,2015-08-10,8.05,0.04,0.0,79.0,69.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Tampa International Airport,Tampa,FL,27.97547,-82.53325,CFMI,18,NaN,False,17
461649,N102UW,Airbus,A320,8,17,1,2058,PHL,BOS,280,AA,American Airlines Inc.,2015-08-17,8.50,0.00,0.0,94.0,73.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Gen. Edward Lawrence Logan International Airport,Boston,MA,42.36435,-71.00518,CFMI,20,NaN,False,17
714096,N102UW,Airbus,A320,8,17,1,1749,MIA,PHL,1013,AA,American Airlines Inc.,2015-08-17,8.95,0.06,0.0,91.0,79.0,Miami International Airport,Miami,FL,25.79325,-80.29056,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,16,NaN,False,17
702394,N102UW,Airbus,A320,8,18,2,1917,BOS,PHL,280,AA,American Airlines Inc.,2015-08-18,8.05,0.14,0.0,87.0,71.0,Gen. Edward Lawrence Logan International Airport,Boston,MA,42.36435,-71.00518,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,07,NaN,False,17
1230124,N102UW,Airbus,A320,8,18,2,1883,PHL,TPA,920,AA,American Airlines Inc.,2015-08-18,7.61,0.00,0.0,92.0,73.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Tampa International Airport,Tampa,FL,27.97547,-82.53325,CFMI,18,31.0,True,17


In [74]:
# only if same day flight condition
df_raw['same_date'] = df_raw.flight_date.dt.date == df_raw.flight_date.dt.date.shift(1)

# only if same aircraft condition 
df_raw['same_plane'] = df_raw.tail_num == df_raw.tail_num.shift(1)

In [75]:
df_raw.head(20)[['flight_date','same_date','same_plane']]

,flight_date,same_date,same_plane
769672,2015-07-21,False,False
1228456,2015-07-21,True,True
708469,2015-07-22,False,True
700195,2015-08-09,False,True
1069062,2015-08-09,True,True
1230079,2015-08-10,False,True
461649,2015-08-17,False,True
714096,2015-08-17,True,True
702394,2015-08-18,False,True
1230124,2015-08-18,True,True


In [76]:
df_raw['previous_delay'] = np.where((df_raw.same_date) & (df_raw.same_plane),df_raw.delayed.shift(1),False)

In [77]:
df_raw.drop(['same_date','same_plane'],axis=1,inplace=True)

In [73]:
df_raw.head(20)

,tail_num,manufacturer,model,month,day,day_of_week,flight_num,orig_airport,dest_airport,distance,airline_code,airline_name,flight_date,avg_wind_orig,precip_orig,snow_orig,max_temp_orig,min_temp_orig,airport_name_orig,city_orig,state_orig,lat_orig,long_orig,airport_dest,city_dest,state_dest,lat_dest,long_dest,engine_manufacturer,hour,total_delay,delayed,aircraft_age,previous_delay
769672,N102UW,Airbus,A320,7,21,2,1829,TPA,PHX,1788,AA,American Airlines Inc.,2015-07-21,8.50,0.01,0.0,89.0,81.0,Tampa International Airport,Tampa,FL,27.97547,-82.53325,Phoenix Sky Harbor International Airport,Phoenix,AZ,33.43417,-112.00806,CFMI,20,NaN,False,17,0
1228456,N102UW,Airbus,A320,7,21,2,575,PHX,TPA,1788,AA,American Airlines Inc.,2015-07-21,6.71,0.00,0.0,106.0,85.0,Phoenix Sky Harbor International Airport,Phoenix,AZ,33.43417,-112.00806,Tampa International Airport,Tampa,FL,27.97547,-82.53325,CFMI,23,NaN,False,17,False
708469,N102UW,Airbus,A320,7,22,3,1789,TPA,PHL,920,AA,American Airlines Inc.,2015-07-22,7.16,0.04,0.0,91.0,80.0,Tampa International Airport,Tampa,FL,27.97547,-82.53325,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,07,NaN,False,17,0
700195,N102UW,Airbus,A320,8,9,7,1850,MCO,PHL,861,AA,American Airlines Inc.,2015-08-09,6.26,0.00,0.0,94.0,74.0,Orlando International Airport,Orlando,FL,28.42889,-81.31603,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,10,NaN,False,17,0
1069062,N102UW,Airbus,A320,8,9,7,1850,PHL,MCO,861,AA,American Airlines Inc.,2015-08-09,4.70,0.00,0.0,86.0,71.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Orlando International Airport,Orlando,FL,28.42889,-81.31603,CFMI,07,NaN,False,17,False
1230079,N102UW,Airbus,A320,8,10,1,1883,PHL,TPA,920,AA,American Airlines Inc.,2015-08-10,8.05,0.04,0.0,79.0,69.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Tampa International Airport,Tampa,FL,27.97547,-82.53325,CFMI,18,NaN,False,17,0
461649,N102UW,Airbus,A320,8,17,1,2058,PHL,BOS,280,AA,American Airlines Inc.,2015-08-17,8.50,0.00,0.0,94.0,73.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Gen. Edward Lawrence Logan International Airport,Boston,MA,42.36435,-71.00518,CFMI,20,NaN,False,17,0
714096,N102UW,Airbus,A320,8,17,1,1749,MIA,PHL,1013,AA,American Airlines Inc.,2015-08-17,8.95,0.06,0.0,91.0,79.0,Miami International Airport,Miami,FL,25.79325,-80.29056,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,16,NaN,False,17,False
702394,N102UW,Airbus,A320,8,18,2,1917,BOS,PHL,280,AA,American Airlines Inc.,2015-08-18,8.05,0.14,0.0,87.0,71.0,Gen. Edward Lawrence Logan International Airport,Boston,MA,42.36435,-71.00518,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,CFMI,07,NaN,False,17,0
1230124,N102UW,Airbus,A320,8,18,2,1883,PHL,TPA,920,AA,American Airlines Inc.,2015-08-18,7.61,0.00,0.0,92.0,73.0,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114,Tampa International Airport,Tampa,FL,27.97547,-82.53325,CFMI,18,31.0,True,17,False


Now I do some feature engineering.  Because a lot of my variables are categorical with high value counts, I needed another way besides one-hot encoding to capture the effect of these features.  For each one, I took the percent of flights delayed for each column value.

In [5]:
delays_by_airport = flights.groupby('orig_airport',as_index=False).agg({'delayed':'mean'}).rename(columns={'delayed': 'airport_delayed'})
flights = flights.merge(delays_by_airport,on='orig_airport')

In [6]:
delays_by_airline = flights.groupby('airline_name',as_index=False).agg({'delayed': 'mean'}).rename(
    columns={'delayed': 'airline_delayed'})
flights = flights.merge(delays_by_airline,on='airline_name')

In [7]:
delays_by_hour = flights.groupby('hour',as_index=False).agg({'delayed': 'mean'}).rename(
    columns={'delayed': 'hour_delayed'})
flights = flights.merge(delays_by_hour,on='hour')

delays_by_month = flights.groupby('month',as_index=False).agg({'delayed': 'mean'}).rename(
    columns={'delayed': 'month_delayed'})
flights = flights.merge(delays_by_month,on='month')

delays_by_weekday = flights.groupby('day_of_week',as_index=False).agg({'delayed': 'mean'}).rename(
    columns={'delayed': 'weekday_delayed'})
flights = flights.merge(delays_by_weekday,on='day_of_week')

delays_by_aircraft_model = flights.groupby('model',as_index=False).agg({'delayed': 'mean'}).rename(
    columns={'delayed': 'model_delayed'})
flights = flights.merge(delays_by_aircraft_model,on='model')

delays_by_engine = flights.groupby('engine_manufacturer',as_index=False).agg({'delayed': 'mean'}).rename(
    columns={'delayed': 'engine_delayed'})
flights = flights.merge(delays_by_engine,on='engine_manufacturer')

delays_by_aircraft = flights.groupby('tail_num',as_index=False).agg({'delayed': 'mean'}).rename(
    columns={'delayed': 'aircraft_delayed'})
flights = flights.merge(delays_by_aircraft,on='tail_num')

In [8]:
flights.head()

,tail_num,manufacturer,model,month,day,day_of_week,flight_num,orig_airport,dest_airport,distance,...,aircraft_age,previous_delay,airport_delayed,airline_delayed,hour_delayed,month_delayed,weekday_delayed,model_delayed,engine_delayed,aircraft_delayed
0,N102UW,Airbus,A320,7,21,2,1829,TPA,PHX,1788,...,17,False,0.187791,0.171255,0.292438,0.231314,0.200221,0.216617,0.191086,0.135593
1,N102UW,Airbus,A320,7,21,2,575,PHX,TPA,1788,...,17,False,0.196470,0.171255,0.220926,0.231314,0.200221,0.216617,0.191086,0.135593
2,N102UW,Airbus,A320,9,1,2,1925,TPA,PHL,920,...,17,False,0.187791,0.171255,0.187565,0.128492,0.200221,0.216617,0.191086,0.135593
3,N102UW,Airbus,A320,9,15,2,1955,PHL,TPA,920,...,17,False,0.198631,0.171255,0.232860,0.128492,0.200221,0.216617,0.191086,0.135593
4,N102UW,Airbus,A320,11,17,2,2058,PHL,BOS,280,...,17,False,0.198631,0.171255,0.292438,0.151843,0.200221,0.216617,0.191086,0.135593


In [9]:
flights.to_csv('flights_final.csv',index=False)